In [1]:
import cv2

import numpy as np
import matplotlib.pyplot as plt
import os
import imageio
from scipy import misc
import glob
owd = os.getcwd()

# Camera Calibration

## Camera calibration is required in order to translate between pixel coordinates and euclidean (real-world) coordinates. The code below uses multiple images from each camera of a checkerboard in order to obtain both information about the camera and the translational error that occurs when one transforms from one coordinate set to another. This approach can be found on the openCV website https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html . Other calibration approaches exist that may be used. 


# Calibration Requirements

## 2-3 second video of a chessboard with known square-size dimensions captured by both cameras. We then take this video and break it down into frames, selecting between 40-60 different frames (can use more if needed for better accuracy, as a larger sample size will yield more accurate results from the algorithm...)
## This post from stackexchange is an excellent guide to achieving consistent, accurate calibration. https://stackoverflow.com/questions/12794876/how-to-verify-the-correctness-of-calibration-of-a-webcam


In [4]:
####### NOTE: Version of code can be found on OPENCV website ##########
###### This code uses checkerboard images from multiple stero images to obtain the camera calibration matrix #####
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.


# PATH in and out here, utilize comments to navigate bash style..
#owd = os.getcwd()
os.chdir(owd)
os.chdir('Cal_a/')


for fname in os.listdir('.'):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# error@gray
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
         # Draw and display the corners
        #img = cv2.drawChessboardCorners(img, (7,6), corners2,ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)
# rendering the chessboard images here, doublecheck this please for error before moving on to the actual calibration...
#cv2.destroyAllWindows()
        
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

print (ret, mtx, dist, rvecs, tvecs)

        


(0.37244664793924753, array([[1.80165734e+03, 0.00000000e+00, 8.21620738e+02],
       [0.00000000e+00, 1.92966064e+03, 5.21154795e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), array([[ 2.17803410e+00, -2.39957837e+01,  4.03266425e-02,
         1.38746289e-01,  1.33204718e+02]]), [array([[ 0.4086959 ],
       [-0.451737  ],
       [ 0.64083758]])], [array([[-7.29836959],
       [-3.8831391 ],
       [43.69335545]])])


In [16]:
mtx = np.hstack(ret)
mtx = np.hstack(rvecs)
mtx = np.hstack(tvecs)
print (mtx)

TypeError: 'float' object is not iterable

In [18]:
##### Calibration saving scheme, let's ask Chris about this and see what he would do! #####
headera='cal_a_'
headerb='cal_b_'
np.savetxt("cal_a_1.csv", mtx, delimiter=",")

## We are now interested in the parameters describing translation, rotation, and camera properties. These are calculated using the calibrateCamera function in openCV.

In [ ]:
#ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
# function returns calibration parameters 
#### parameters that we are interested in: tvecs, rvecs, mtx #####


# Error in Coordinates
## We can reproject and find the error in our transforms. This should be as close to 0 as possible (obviously). Following code calculates total error, this will be modified in the future to give error in each axis (hopefully..).


In [10]:
mean_error = 0
tot_error = 0
for i in xrange(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i],imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    tot_error += error

print "total error: ", mean_error/len(objpoints)

total error:  0


# Calculating the fundamental matrix (optional, will most likely be removed in final version...)

## In order to calculate the fundamental matrix, multiple images from both stereo scenes must be taken. ROI's are then matched across the images using an algorithm (typically 8point or RANSAC). OpenCV has some documentation on this https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_epipolar_geometry/py_epipolar_geometry.html and there are lots of sources online that go very deeply into epipolar geometry and stereo reconstruction using fundamental matrices.
## Additionally, point pair matching can be done using SIFT/FLANN implementation, as described in the openCV documentation! This is an automated procedure that saves the step of manually labelling points by hand. This procedure should be used in a large-scale automated workflow.

In [ ]:
######### NOTE: Version of code can be found on OPENCV website #############
######### This code does not mark ROI's to match, so it cannot be used to translate anything into real world coordinates #######

s1 = cv2.imread('002533.png',0)  #head-on image.
s2 = cv2.imread('cv2.png',0) #direct opposite image.


# note: you need openCV-contrib library installed as well in order to run SIFT/SURF

sift = cv2.xfeatures2d.SIFT_create()
# loads SIFT algorithm, docs are on OPENCV

# find keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(s1,None)
kp2, des2 = sift.detectAndCompute(s2,None)

# FLANN parameters
# FLANN documentation at xxxxxx
FLANN_INDEX_KDTREE = 0
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)

good = []
pts1 = []
pts2 = []

# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        good.append(m)
        pts2.append(kp2[m.trainIdx].pt)
        pts1.append(kp1[m.queryIdx].pt)
        
        

        
        
        
        
pts1 = np.int32(pts1)
pts2 = np.int32(pts2)
F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.FM_LMEDS)

# We select only inlier points
pts1 = pts1[mask.ravel()==1]
pts2 = pts2[mask.ravel()==1]

# Translation and Rotation Matrices
## We need to obtain the translation and rotation matrices for our system. This can be done using RANSAC and openCV https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_pose/py_pose.html. There are other methods as well that leverage vector equations and SVD algorithms to estimate pose parameters, but these can be unwieldy. 


In [7]:
########## As always, code is on OpenCV ############
######### Ensure that image pairs are matched this way ######
def draw(img, corners, imgpts):
    corner = tuple(corners[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

os.chdir(owd)
os.chdir('Cal')

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)


# adjust directory here into directory of local frames from video.
for fname in glob.glob('left*.jpg'):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7,6),None)

    if ret == True:
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)

        # Find the rotation and translation vectors.
        rvecs, tvecs, inliers = cv2.solvePnPRansac(objp, corners2, mtx, dist)

        # project 3D points to image plane
        imgpts, jac = cv2.projectPoints(axis, rvecs, tvecs, mtx, dist)

        #img = draw(img,corners2,imgpts)
        #cv2.imshow('img',img)
        #k = cv2.waitKey(0) & 0xff
        #if k == 's':
         #   cv2.imwrite(fname[:6]+'.png', img)

cv2.destroyAllWindows()

#print (rvecs,tvecs,inliers)





# 3-D Coordinates

## Now that we have both essential matrices and camera calibration information in hand, we can apply transforms to our pixel images. Among other things, we can translate between euclidean (real world) coordinates, as well as perform simple geometric transforms (rotation etc).
## These transforms are represented by the terms P'1 = K1 * P1 and P'2 = P2 * K2. We can then use triangulation to map each pixel coordinate to the appropriate coordinates in 3 dimensions. Since P1 is our origin camera, it is the identity matrix. P2 is given by R|T, found above. Triangulation method refined from http://www.morethantechnical.com/2012/01/04/simple-triangulation-with-opencv-from-harley-zisserman-w-code/. 
## Following code translates DLC coordinates that have been labeled into 3-D euclidean coordinates. The following code will be updated to be fully integrated with DLC (another task in of itself...)


In [1]:
R_T = np.hstack(rvecs,tvecs) # stack into vector for eq.

p_1 = K_1 * np.identity(3,3)
p_2 = K_2 * R_T


#### Now Perform Triangulation for points.. see ####
def true_coords(projMatr1,projMatr2,projPoints1,projPoints2):
    # define true_points array? 
    true_points = cv2.triangulatePoints(projMatr1, projMatr2, projPoints1, projPoints2)
    return true_points /= true_points[3] #return normalized coordinates...


### Triangulation technique is essential for translating DLC coordinates (labeled) to our 3-D coordinates ###




#output /= output[3] # normalizes new coordinates by depth term

NameError: name 'np' is not defined

# DLC Coordinates and our Transform
When DeepLabCut labels our ROI for a given video, it produces a dataframe containing pixel coordinates for a given ROI over frames. Each coordinate has both predictive liklihood value (alpha value) as well as pixel coordinate predicted by DLC. 
### Let's load in a sample dataframe for two features. 

In [2]:
import pandas as pd
filename = 'handle_videoDeepCut_resnet50_Demo2Jan30shuffle1_150000.h5'
pd.read_hdf(filename)

scorer    DeepCut_resnet50_Demo2Jan30shuffle1_150000                         \
bodyparts                                    Handle2                          
coords                                             x           y likelihood   
0                                         459.698007  644.095077   0.984066   
1                                         459.659744  643.862641   0.898935   
2                                         460.157639  645.001269   0.927753   
3                                         460.184307  644.414764   0.922327   
4                                         460.123447  644.362911   0.949559   
5                                         460.309819  644.382935   0.940193   
6                                         460.337280  643.989951   0.894790   
7                                         460.058171  644.032325   0.931594   
8                                         460.091916  643.665556   0.823999   
9                                         460.179054  643.873811   0.859642   
10                                        460.111809  644.105684   0.948210   
11                                        460.162704  644.058620   0.949083   
12                                        460.775349  645.611871   0.744201   
13                                        460.670418  644.997239   0.553514   
14                                        460.853478  644.946663   0.500921   
15                                        460.737954  644.620835   0.594497   
16                                        460.731504  644.563834   0.603671   
17                                        460.598934  644.974773   0.604118   
18                                        460.703841  644.940382   0.595788   
19                                        460.807677  644.682933   0.469658   
20                                        460.496166  644.493997   0.639191   
21                                        460.636905  644.217780   0.552394   
22                                        460.797839  644.342192   0.585696   
23                                        460.809218  644.340718   0.583028   
24                                        459.609526  644.069057   0.526094   
25                                        459.620590  644.222333   0.607410   
26                                        459.369897  643.855023   0.641267   
27                                        459.938687  644.110054   0.580519   
28                                        459.909480  643.969595   0.579831   
29                                        459.905756  644.107380   0.526046   
..                                               ...         ...        ...   
404                                       980.662813  628.499221   0.998133   
405                                       980.685695  628.432787   0.998135   
406                                       980.700560  628.597613   0.998320   
407                                       980.736422  628.630321   0.998552   
408                                       980.170480  628.393937   0.995241   
409                                       980.060585  628.367883   0.994804   
410                                       980.181692  628.556347   0.995269   
411                                       980.117931  628.477179   0.994959   
412                                       980.228479  628.527971   0.996030   
413                                       980.184663  628.622011   0.996345   
414                                       980.171736  628.585748   0.996118   
415                                       980.164472  628.568655   0.995761   
416                                       980.150071  628.569304   0.995883   
417                                       980.158342  628.490255   0.995930   
418                                       980.232054  628.258653   0.996714   
419                                       980.225272  628.337324   0.996728   
420                                       979.8327

# So for each frame, we have coordinate pairs for each ROI we select!
These coordinates are in pixel-space however. Also of note, camera 1 and camera 2 are not defined by DLC. We must select which coordinate pair belongs to which, normalize the coordinates to be matched up with camera pixelspace, then transform these coordinate pairs into 3-D coordinates.
# Workflow
### Step 1
Select which coordinate pair belongs to Camera 1 or 2
### Step 2
Normalize the coordinate pairs (this is just simple algebra done below)
### Step 3
Apply the transform uniformly to our pairs
### Step 4 
Visualize coordinate pairs in Euclidean space

In [ ]:
#### Normalization #####
# Find ROI with x coordinates above certain limit (X/2, where X is total length of image processed through DLC)
# Select ROI's with these coordinates, subtract X/2 from their x pixel coordinates to reframe
